In [0]:
# Cell 0: Imports
import os
import uuid
import json
import base64
import pandas as pd
from io import BytesIO
from datetime import datetime
from pyspark.sql import functions as F
from pyspark.sql.functions import lit, current_timestamp
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType
from delta.tables import DeltaTable

from clinical_data_standards_framework.utils import (
    save_with_audit,
    find_excel_header_row,
    get_header_filter_patterns,
    parse_max_length,
    is_required_field,
    is_transfer_key_field,
    get_transfer_metadata_flexible_mapper
)

In [ ]:
# Cell 1: Check if Previous Task Found Documents
# Skip this task if extract_excel_sheets found no documents

try:
    documents_found = dbutils.jobs.taskValues.get(taskKey="extract_excel_sheets", key="documents_found")
    
    if documents_found == "false":
        print("⏭️  No documents were found in previous task (extract_excel_sheets)")
        print("   Skipping transfer metadata processing.")
        dbutils.notebook.exit("Skipped - no documents to process")
    
    print(f"✓ Documents found in previous task: {documents_found}")
    
except Exception as e:
    # If task value doesn't exist, assume we should continue (backwards compatibility)
    print(f"⚠ Could not read 'documents_found' task value: {e}")
    print("  Continuing anyway (backwards compatibility)")
    pass


In [0]:
# Cell 1: Get Configuration from Setup Task
globals_dict = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="globals"))
pipeline_config = json.loads(dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config"))
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")

# Extract needed values from globals
catalog = globals_dict['catalog']
bronze_schema = globals_dict['bronze_schema']
silver_schema = globals_dict['silver_schema']

print(f"Catalog: {catalog}")
print(f"Bronze Schema: {bronze_schema}")
print(f"Silver Schema: {silver_schema}")
print(f"Created by: {created_by_principal}")
print(f"Job ID: {databricks_job_id}, Job Name: {databricks_job_name}, Run ID: {databricks_run_id}")

In [0]:
# Cell 2: Construct Table Names
sheets_table = f"{catalog}.{bronze_schema}.md_dta_excel_file_raw"
history_table = f"{catalog}.{bronze_schema}.md_dta_history"
status_table = f"{catalog}.{bronze_schema}.md_document_status"
output_table = f"{catalog}.silver_md.md_transfer_variable_field_normalized"  # Target normalized silver table

print(f"Sheets table: {sheets_table}")
print(f"History table: {history_table}")
print(f"Status table: {status_table}")
print(f"Output table: {output_table}")

In [0]:
# Cell 3: Read Transfer Metadata Sheets with Parent Content
# Join sheet metadata with parent document content (base64)
# Filter out already successfully processed sheets (row_status = COMPLETED)

print(f"{'='*80}")
print(f"Reading transfer_metadata sheets...")
print(f"{'='*80}")

# Read sheet metadata filtered for transfer_metadata category
# Include trial_id, data_stream_type, data_provider_name for composite PK
df_sheets = spark.table(sheets_table).filter(
    F.col("sheet_category") == "transfer_metadata"
).select(
    "document_id",
    "parent_document_id",
    "sheet_name",
    "source_file_path",
    "trial_id",
    "data_stream_type",
    "data_provider_name"
)

total_sheets = df_sheets.count()
print(f"Total transfer_metadata sheets in source: {total_sheets}")

# Check which sheets have already been processed (row_status IS NOT NULL)
if spark.catalog.tableExists(output_table):
    # Read already processed sheets (any trial/stream/provider with non-null row_status)
    # This includes both COMPLETED and MANUAL_REVIEW_REQUIRED/ERROR records
    df_already_processed = spark.table(output_table).filter(
        F.col("row_status").isNotNull()
    ).select(
        "trial_id",
        "data_stream_type", 
        "data_provider_name"
    ).distinct()
    
    # Anti-join to exclude already processed sheets
    # Only process sheets that have NO records in the output table yet
    df_sheets_to_process = df_sheets.join(
        df_already_processed,
        on=["trial_id", "data_stream_type", "data_provider_name"],
        how="left_anti"
    )
    
    already_processed_count = total_sheets - df_sheets_to_process.count()
    print(f"Sheets already processed (any status): {already_processed_count}")
    print(f"Sheets to process (row_status IS NULL): {df_sheets_to_process.count()}")
else:
    print(f"Output table does not exist yet - will create it")
    df_sheets_to_process = df_sheets

# Join with parent documents to get content_base64
df_with_content = df_sheets_to_process.join(
    spark.table(history_table).select(
        F.col("document_id").alias("parent_id"),
        "content_base64"
    ),
    df_sheets_to_process.parent_document_id == F.col("parent_id")
)

sheet_count = df_with_content.count()
print(f"Sheets to process in this run: {sheet_count}")
print(f"{'='*80}")

if sheet_count == 0:
    print("✓ No new transfer_metadata sheets to process. All up-to-date.")
    # Set task values for downstream tasks
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value="UP_TO_DATE")
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value="0")
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value="0")
    dbutils.notebook.exit("All transfer_metadata sheets already processed")

In [0]:
# Cell 4: Define Output Schema
# Schema for normalized transfer variable fields table
# Composite PK: (trial_id, data_stream_type, data_provider_name, transfer_variable_name)

output_schema = StructType([
    StructField("transfer_variable_id", StringType(), False),  # PK - UUID
    StructField("trial_id", StringType(), False),
    StructField("data_stream_type", StringType(), False),
    StructField("data_provider_name", StringType(), False),
    StructField("transfer_variable_name", StringType(), False),
    StructField("codelist_id", StringType(), True),  # FK - populated later
    StructField("transfer_variable_label", StringType(), True),
    StructField("variable_description", StringType(), True),
    StructField("transfer_variable_order", IntegerType(), True),
    StructField("format", StringType(), True),
    StructField("populate_for_all_records", BooleanType(), True),
    StructField("transfer_file_key", BooleanType(), True),
    StructField("example_values", StringType(), True),
    StructField("categorization_notes", StringType(), True),
    StructField("row_status", StringType(), True)
])

print(f"Output schema defined with {len(output_schema.fields)} fields")
print(f"Primary Key: transfer_variable_id (UUID)")
print(f"Natural Key: (trial_id, data_stream_type, data_provider_name, transfer_variable_name)")


In [0]:
# Cell 5: Process Transfer Metadata Sheets
# Parse each sheet and extract transfer variable fields

print(f"{'='*80}")
print(f"Processing transfer metadata sheets...")
print(f"{'='*80}")

all_records = []

for row in df_with_content.collect():
    sheet_name = row['sheet_name']
    source_path = row['source_file_path']
    content_b64 = row['content_base64']
    
    # Metadata for natural key
    trial_id = row['trial_id']
    data_stream_type = row['data_stream_type']
    data_provider_name = row['data_provider_name']
    
    print(f"\nProcessing: {sheet_name}")
    print(f"  Trial: {trial_id}, Stream: {data_stream_type}, Provider: {data_provider_name}")
    
    try:
        # Decode parent Excel file from base64
        excel_bytes = base64.b64decode(content_b64)
        
        # Detect engine based on file extension
        file_ext = os.path.splitext(source_path.lower())[1]
        engine = 'xlrd' if file_ext in ['.xls', '.xlt'] else 'openpyxl'
        
        # Read sheet to find header row
        raw_df = pd.read_excel(BytesIO(excel_bytes), sheet_name=sheet_name, header=None, engine=engine)
        header_row = find_excel_header_row(raw_df, [
            r'transfer.*variable.*name',
            r'format'
        ])
        
        # Read sheet with proper header
        df_sheet = pd.read_excel(
            BytesIO(excel_bytes),
            sheet_name=sheet_name,
            header=header_row,
            engine=engine
        ).dropna(axis=1, how='all')
        
        print(f"  Original columns: {list(df_sheet.columns)[:5]}...")  # Show first 5
        
        # Use flexible column mapper
        col_map, missing_required = get_transfer_metadata_flexible_mapper(list(df_sheet.columns))
        
        print(f"  Mapped {len(col_map)} columns")
        
        # Check for missing required columns
        sheet_status = None
        sheet_notes = None
        if missing_required:
            sheet_status = "MANUAL_REVIEW_REQUIRED"
            sheet_notes = f"Missing required columns: {', '.join(missing_required)}"
            print(f"  ⚠ WARNING: {sheet_notes}")
        
        # Rename columns using flexible mapper
        df_sheet = df_sheet.rename(columns=col_map)
        
        # Filter to rows with transfer_variable_name
        if "transfer_variable_name" not in df_sheet.columns:
            print(f"  ✗ No transfer_variable_name column found, skipping sheet")
            continue
            
        df_sheet = df_sheet[df_sheet["transfer_variable_name"].notna()].copy()
        
        # Debug: Show first 3 rows before header filtering
        print(f"  First 3 rows before header filtering:")
        if len(df_sheet) > 0:
            for idx in range(min(3, len(df_sheet))):
                row_data = df_sheet.iloc[idx]
                var_name = row_data.get('transfer_variable_name', 'N/A')
                file_order = row_data.get('transfer_file_order', 'N/A')
                print(f"    Row {idx}: transfer_variable_name='{var_name}', transfer_file_order='{file_order}'")
        
        # Get header patterns for transfer_metadata sheets
        header_patterns = get_header_filter_patterns('transfer_metadata')
        
        # Check each column for header text and create a combined mask
        header_mask = pd.Series([False] * len(df_sheet), index=df_sheet.index)
        
        for col, patterns in header_patterns.items():
            if col in df_sheet.columns:
                for pattern in patterns:
                    mask = df_sheet[col].astype(str).str.lower().str.contains(
                        pattern, na=False, regex=True
                    )
                    if mask.any():
                        print(f"    Found header pattern '{pattern}' in column '{col}': {mask.sum()} rows")
                    header_mask = header_mask | mask
        
        # Remove header rows
        if header_mask.any():
            header_row_count = header_mask.sum()
            print(f"    ⚠ Removing {header_row_count} header-like row(s) from data")
            print(f"    Original row count: {len(df_sheet)}")
            df_sheet = df_sheet[~header_mask].copy()
            print(f"    Remaining rows: {len(df_sheet)}")
        else:
            print(f"    ✓ No header rows detected in data")
        
        print(f"  Found {len(df_sheet)} transfer variables")
        
        # Parse each variable row
        for idx, r in df_sheet.iterrows():
            # Determine row-level status
            row_status = sheet_status  # Inherit sheet-level issues
            row_notes = sheet_notes
            
            # Check for missing required values in this row
            row_missing = []
            for req_col in ['transfer_variable_name', 'transfer_file_order', 'data_type_format', 
                            'anticipated_max_length_raw', 'transfer_file_key', 'populate_for_all_records_raw']:
                if req_col in df_sheet.columns:
                    val = r.get(req_col)
                    if pd.isna(val) or (isinstance(val, str) and str(val).strip() == ''):
                        row_missing.append(req_col)
                else:
                    # Column not in sheet (already flagged at sheet level)
                    pass
            
            if row_missing and not row_status:
                row_status = "MANUAL_REVIEW_REQUIRED"
                row_notes = f"Missing required values: {', '.join(row_missing)}"
            
            record = {
                # Primary Key (UUID)
                'transfer_variable_id': str(uuid.uuid4()),
                
                # Natural Key
                'trial_id': trial_id,
                'data_stream_type': data_stream_type,
                'data_provider_name': data_provider_name,
                'transfer_variable_name': str(r.get('transfer_variable_name', '')),
                
                # FK (populated later)
                'codelist_id': None,
                
                # Variable attributes
                'transfer_variable_label': str(r.get('transfer_variable_label')) if pd.notna(r.get('transfer_variable_label')) else None,
                'variable_description': str(r.get('variable_description')) if pd.notna(r.get('variable_description')) else None,
                'transfer_variable_order': int(float(r.get('transfer_file_order'))) if pd.notna(r.get('transfer_file_order')) and str(r.get('transfer_file_order')).replace('.','',1).replace('-','',1).isdigit() else None,
                'format': str(r.get('data_type_format')) if pd.notna(r.get('data_type_format')) else None,
                
                # Boolean flags (handles pandas NaN properly)
                'populate_for_all_records': is_required_field(r.get('populate_for_all_records_raw')),
                'transfer_file_key': is_transfer_key_field(r.get('transfer_file_key')),
                
                # Optional fields
                'example_values': str(r.get('example_values')) if pd.notna(r.get('example_values')) else None,
                
                # Error tracking
                'categorization_notes': row_notes,
                'row_status': row_status
            }
            
            all_records.append(record)
        
        print(f"  ✓ Processed {len(df_sheet)} variables")
        
    except Exception as e:
        print(f"  ✗ ERROR processing sheet: {str(e)}")
        import traceback
        traceback.print_exc()
        continue

print(f"\n{'='*80}")
print(f"✓ Processing complete: {len(all_records)} total records")
print(f"{'='*80}")


In [0]:
# Cell 6: Write Results to Silver Table & Set Task Values
# Create DataFrame, write with audit columns, and signal success/failure to downstream tasks

if len(all_records) == 0:
    print("No records to write. Exiting.")
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value="NO_DATA")
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value="0")
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value="0")
    dbutils.notebook.exit("No transfer variables found to process")

# Count success vs error records
success_count = len([r for r in all_records if r.get('row_status') == 'COMPLETED'])
error_count = len([r for r in all_records if r.get('row_status') and r['row_status'] != 'COMPLETED'])

print(f"\n{'='*80}")
print(f"Processing Summary:")
print(f"  Total records: {len(all_records)}")
print(f"  Success (COMPLETED): {success_count}")
print(f"  Errors/Manual Review: {error_count}")
print(f"{'='*80}")

# Create Spark DataFrame
df_result = spark.createDataFrame(all_records, schema=output_schema)

# Display sample
print(f"\nSample records:")
df_result.show(5, truncate=False)

# Write to silver table with audit columns using MERGE mode for incremental updates
print(f"\nWriting {len(all_records)} records to {output_table}...")
save_with_audit(
    df=df_result,
    table_name=output_table,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode='merge',  # Use merge for incremental updates
    merge_keys=['trial_id', 'data_stream_type', 'data_provider_name', 'transfer_variable_name']
)

print(f"\n{'='*80}")
print(f"✓ Successfully wrote {len(all_records)} records")
print(f"  Table: {output_table}")
print(f"  Mode: merge (incremental updates)")
print(f"  Merge keys: trial_id, data_stream_type, data_provider_name, transfer_variable_name")
print(f"  Audit columns added automatically")
print(f"{'='*80}")

# Set task values for conditional downstream execution
if success_count == 0 and error_count > 0:
    # All records failed - downstream tasks should not run
    task_status = "FAILED"
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value=task_status)
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value=str(error_count))
    raise Exception(f"All {error_count} transfer variable records failed. Check categorization_notes for details.")
elif success_count > 0:
    # At least some success - downstream can proceed
    task_status = "SUCCESS"
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value=task_status)
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value=str(success_count))
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value=str(error_count))
    print(f"\n✓ Task status: {task_status}")
    print(f"  Downstream tasks will proceed.")
else:
    # No records at all (shouldn't reach here due to earlier exit, but handle it)
    task_status = "NO_DATA"
    dbutils.jobs.taskValues.set(key="transfer_variables_status", value=task_status)
    dbutils.jobs.taskValues.set(key="transfer_variables_success_count", value="0")
    dbutils.jobs.taskValues.set(key="transfer_variables_error_count", value="0")

In [0]:
# Cell 7: Verify Results
# Query the table to verify the write

print(f"\nVerifying table: {output_table}")
result_count = spark.table(output_table).count()
print(f"Total records in table: {result_count}")

# Show sample
print(f"\nSample records from {output_table}:")
spark.table(output_table).select(
    "transfer_variable_id",
    "trial_id",
    "transfer_variable_name",
    "transfer_variable_order",
    "populate_for_all_records",
    "transfer_file_key",
    "example_values",
    "row_status",
    "categorization_notes",
    "databricks_job_name"
).orderBy("trial_id", "transfer_variable_order").show(10, truncate=False)